In [ ]:
!pip install multidict
!pip install -U spacy
!python -m spacy download fr_core_news_sm
!python -m spacy download fr_dep_news_trf

## Ne pas oublier de relancer l'environnement

In [ ]:
import multidict as multidict

In [ ]:

import os

from os import path
from collections import Counter
import pickle
import re 
import spacy

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Traitement des fichiers produits par le scrapper
> ndf est le fichier produit par le scapper. On le charge le spacytraite pour visualiser

In [ ]:
import pandas as pd
import os
stats = dict()
for ndf in os.listdir("/content/drive/MyDrive/Com-Web/Projet Master2 Langues/OUT/ContenusSites/"):
  if not ndf.startswith('.'):
    fichierEntree = '/content/drive/MyDrive/Com-Web/Projet Master2 Langues/OUT/ContenusSites/' +ndf
    with open (fichierEntree, 'rb') as  fictemp: #pour ouvrir les contenus 
      soup=pickle.load(fictemp)

    data  = [len(soup [url]) for url in soup.keys()]
    donnees = pd.DataFrame.from_dict(data)
    stats [ndf .replace('.pkl', '')] = donnees .describe()

 # stats contient la description statistique des données fondée sur la taille
 # de celle-ci. Cette variable servira a exclure les url dont la variation
 # à la moyenne est trop importante : trop ou pas assez sont considérés comme
 # des singularités. Cela peut provenir de la forme choisie pour la page (grande
 # page qui contient "tout" avec un ascenceur interminable ou page composée
 # par des appels url en js que le collecteur ne reconstruit pas) 

In [ ]:
stats

In [ ]:
# suppression des extrêmes
seuil = 2.5
# le code qui suit sert à exclure les extrêmes (taille de collecte) hors zone 
# entre la valeur moyenne sur la collecte divisé par le seuil
# et cette valeur multipliée par le seuil
# il faut réitérer la boucle plusieurs fois jusqu'à convergence (le nombre d'exclus ne varie plus)

# Si le seuil est modifié il faut ré-exécuter la cellule précédente
aGarder, aVirer = dict(), dict()
for ndf in os.listdir("/content/drive/MyDrive/Com-Web/Projet Master2 Langues/OUT/ContenusSites/"):
  if not ndf.startswith('.'):
    
    fichierEntree = '/content/drive/MyDrive/Com-Web/Projet Master2 Langues/OUT/ContenusSites/' +ndf
    with open (fichierEntree, 'rb') as  fictemp: #pour ouvrir les contenus 
      soup=pickle.load(fictemp)
    ndf = ndf .replace('.pkl', '')
    if ndf not in aGarder .keys():
      aGarder [ndf] = []
    if ndf not in aVirer .keys():
      aVirer [ndf] = []
    for url in soup .keys():
      if url not in aVirer [ndf]:
        if len(soup [url]) > stats [ndf] [0]['mean']/seuil:
          if len(soup [url]) < stats [ndf] [0]['mean']*seuil:
            aGarder [ndf].append(url)
          else:
            aVirer [ndf].append(url)         
        else:
            aVirer [ndf].append(url) 
    print (ndf, "  ", stats [ndf] [0]['count'], " exclus ", len(aVirer [ndf]), aVirer [ndf])

    data  = [len(soup [url]) for url in soup.keys() if url not in aVirer[ndf]]
    # mise à jour des stats
    
    donnees = pd.DataFrame.from_dict(data)
    stats [ndf .replace('.pkl', '')] = donnees .describe()


parcs    9.0  exclus  1 ['http://www.ecrins-parcnational.fr/']
organisations    12.0  exclus  9 ['http://ademe.fr', 'http://ffcam.fr', 'http://ffrandonnee.fr', 'http://fondation-patrimoine.org', 'http://iucn.org', 'http://ffem.fr', 'http://eaurmc.fr', 'http://atout-france.fr', 'http://pifh.fr']
oTourism    13.0  exclus  11 ['http://tourismepaca.fr', 'http://lepradet-tourisme.fr', 'http://paysdefayence.com', 'http://provence-alpes-cotedazur.com', 'http://gassin.eu', 'http://ramatuelle-tourisme.com', 'http://sanary-tourisme.com', 'http://gap-tourisme.fr', 'http://tourisme-ouestvar.com', 'http://saintcyrsurmer.com', 'http://sainte-maxime.com']
Services    68.0  exclus  45 ['http://grand-tour-ecrins.fr', 'http://aquafamily.fr', 'https://www.kiddyparc.com/', 'http://balade-en-mer-83.fr', 'http://thalassa.com', 'http://toulon-hyeres.aeroport.fr', 'http://clubkayaklepradet.fr', 'http://plongee-var.fr', 'http://fracpaca.org', 'http://group-indigo.com', 'http://ada.fr', 'http://art-et-vin.net',

In [ ]:
fichierEntree = '/content/drive/MyDrive/Com-Web/Projet Master2 Langues/OUT/ContenusSites/' +"organisations.pkl"
with open (fichierEntree, 'rb') as  fictemp: #pour ouvrir les contenus 
      soup=pickle.load(fictemp)
soup ['http://pifh.fr']

In [ ]:
import pprint as pp
pp.pprint(stats)

{'Gouv':                  0
count     5.000000
mean   1263.400000
std     334.365519
min     965.000000
25%     965.000000
50%    1151.000000
75%    1568.000000
max    1668.000000,
 'Hebergement':                  0
count    73.000000
mean   2670.616438
std    1329.418750
min    1072.000000
25%    1674.000000
50%    2155.000000
75%    3432.000000
max    6338.000000,
 'Services':                  0
count    68.000000
mean   2449.014706
std    1265.738985
min     989.000000
25%    1522.000000
50%    2140.000000
75%    3070.000000
max    5702.000000,
 'locations':                  0
count    26.000000
mean   2329.115385
std     972.180943
min     999.000000
25%    1586.750000
50%    2193.500000
75%    2917.750000
max    5016.000000,
 'oTourism':                  0
count    13.000000
mean   3325.000000
std    1301.210654
min    1601.000000
25%    2026.000000
50%    3491.000000
75%    4612.000000
max    5211.000000,
 'organisations':                  0
count    12.000000
mean   1535.750000


In [ ]:
tempVerb = dict() # fonction du ndf
tempNoun = dict()
tempAdj = dict()
tempLoc = dict()
tempMisc = dict()
tempPer = dict()
tempOrg = dict()
dicoEtiquettage = dict()
aEviter = dict()
verbes = dict() #listes à remplir avec verbes/adj/noms/entités trouvés 
adjectifs  = dict() 
noms = dict()

adverbes = dict()
tempAdv = dict()

In [ ]:

for ndf in os.listdir("/content/drive/MyDrive/Com-Web/Projet Master2 Langues/OUT/ContenusSites/"):
  fichierEntree = '/content/drive/MyDrive/Com-Web/Projet Master2 Langues/OUT/ContenusSites/' +ndf
  if not ndf.startswith('.'):
    # chargement des données collectées par le scrapeur
    with open (fichierEntree, 'rb') as  fictemp: #pour ouvrir les contenus 
      soupe=pickle.load(fictemp)
    ndf = ndf .replace('.pkl', '')
    
    # nettoyage
    tousLesContenus = """""" # une longue chaine qui contiendra tous les contenus de la ndf
    for url in aGarder [ndf]:
      tousLesContenus += soupe [url] + "\n" 
    # deux modèles de données spacy sont utilisés du fait de leur efficacité relative à la détection des entités
    # 'fr_core_news_sm' fonctionne mieux pour la détection des entités
    tal = spacy.load('fr_core_news_sm')
    # 'fr_dep_news_trf' fonctionne mieux pour le reste
    tal1 = spacy.load('fr_dep_news_trf')
    doc = tal(str(tousLesContenus.lower()))

    # Si l'environnement n'a pas été redémarré, le code plante ici
    dicoEtiquettage [ndf] = dict()

    entites = []
    for entity in doc.ents:
      if entity.label_ not in dicoEtiquettage [ndf] .keys():
        dicoEtiquettage [ndf] [entity.label_] = []
        dicoEtiquettage [ndf] [entity.label_] .append(entity.text)
        entites.append(entity.label_)
      else:
        dicoEtiquettage [ndf] [entity.label_] .append(entity.text)
      #entites.append(entity.text)
      #dicoEtiquettage [ndf] [entity.label_] = entites #dico: [étiquettage de entité donné par spacy] = la liste des entités
    if 'LOC' in dicoEtiquettage [ndf] .keys():
        aEviter [ndf] = dicoEtiquettage [ndf] ['LOC']
    if 'MISC' in dicoEtiquettage [ndf] .keys():
        aEviter [ndf] += dicoEtiquettage [ndf] ['MISC']
    if 'PER' in dicoEtiquettage [ndf] .keys():
        aEviter [ndf] += dicoEtiquettage [ndf] ['PER']
    if 'ORG' in dicoEtiquettage [ndf] .keys():
        aEviter [ndf] += dicoEtiquettage [ndf] ['ORG']
      # utilisation de l'autre modèle
    doc = tal1(str(tousLesContenus).lower())

    if ndf not in verbes.keys():
        verbes [ndf] = []
    if ndf not in adjectifs.keys():
        adjectifs [ndf] = []
    if ndf not in noms.keys():
        noms [ndf] = []
    
    if ndf not in adverbes.keys(): #ADVERBES
        adverbes [ndf] = []
    for truc in doc: 
        if truc.text not in aEviter [ndf] and truc.text.lower() not in aEviter [ndf]: 
          if len(truc.text)>2:
            if truc.pos_ == "VERB":
              verbes [ndf].append(truc.lemma_) #ajoute à la liste (.lemma_ fait la lemmatisation)
              dicoEtiquettage [ndf][truc.pos_] = verbes [ndf] #ajoute dans le dico: ["VERB"] = la liste des verbes
            if truc.pos_ == "ADJ":
              adjectifs [ndf].append(truc.lemma_) 
              dicoEtiquettage [ndf][truc.pos_] = adjectifs [ndf] #dico:["ADJ"] = la liste des adjectifs
            if truc.pos_ == "NOUN":
              noms [ndf].append(truc.lemma_)
              dicoEtiquettage [ndf][truc.pos_] = noms [ndf] #dico: ["NOUN"] = la liste des noms  
            
            if truc.pos_ == "ADV": 
              adverbes [ndf].append(truc.lemma_)
              dicoEtiquettage [ndf][truc.pos_] = adverbes [ndf] #dico: ["ADV"] = la liste des adverbes

    for cle in dicoEtiquettage [ndf]: 
        if cle == "VERB":
          tempVerb [ndf] = Counter(dicoEtiquettage [ndf][cle])
        if cle == "NOUN":
          tempNoun [ndf] = Counter(dicoEtiquettage [ndf][cle])
        if cle == "ADJ":
          tempAdj [ndf] = Counter(dicoEtiquettage [ndf][cle])
        if cle == "LOC":
          tempLoc [ndf] = Counter(dicoEtiquettage [ndf][cle])
        if cle == "MISC":
          tempMisc [ndf] = Counter(dicoEtiquettage [ndf][cle])
        if cle == "PER":
          tempPer [ndf]= Counter(dicoEtiquettage [ndf][cle])
        if cle == "ORG":
          tempOrg [ndf]= Counter(dicoEtiquettage [ndf][cle])
   
        if cle == "ADV":
          tempAdv [ndf]= Counter(dicoEtiquettage [ndf][cle])
    

# Sauvegarde des traitements

In [ ]:
fichierDeSortie = '/content/drive/MyDrive/Com-Web/Projet Master2 Langues/OUT/ContenusTraites/' +'spacySites.pkl'
with open (fichierDeSortie, 'wb') as  fictemp: #on met tous les contenus dans pickle
  pickle.dump(dicoEtiquettage, fictemp)


In [ ]:
fichierDeSortie = '/content/drive/MyDrive/Com-Web/Projet Master2 Langues/OUT/ContenusTraites/' +'stats.pkl'
with open (fichierDeSortie, 'wb') as  fictemp: #on met tous les contenus dans pickle
  pickle.dump(stats, fictemp)


In [ ]:
fichierDeSortie = '/content/drive/MyDrive/Com-Web/Projet Master2 Langues/OUT/ContenusTraites/' +"verbesSites.pkl"
with open (fichierDeSortie, 'wb') as  fictemp: #on met tous les contenus dans pickle
  pickle.dump(verbes, fictemp)

fichierDeSortie = '/content/drive/MyDrive/Com-Web/Projet Master2 Langues/OUT/ContenusTraites/' +'nomsSites.pkl'
with open (fichierDeSortie, 'wb') as  fictemp: #on met tous les contenus dans pickle
  pickle.dump(noms, fictemp)

fichierDeSortie = '/content/drive/MyDrive/Com-Web/Projet Master2 Langues/OUT/ContenusTraites/' +'adjectifsSites.pkl'
with open (fichierDeSortie, 'wb') as  fictemp: #on met tous les contenus dans pickle
  pickle.dump(adjectifs, fictemp)

fichierDeSortie = '/content/drive/MyDrive/Com-Web/Projet Master2 Langues/OUT/ContenusTraites/' +'adverbesSites.pkl'
with open (fichierDeSortie, 'wb') as  fictemp: #on met tous les contenus dans pickle
  pickle.dump(adverbes, fictemp)

In [ ]:
dicoEtiquettage .keys()

dict_keys(['Hebergement', 'restauration', 'locations', 'parcs', 'organisations', 'oTourism', 'Services', 'Gouv'])